In [ ]:
## Gravity model to predict the intercity medical patient flow, affected by the relative relationship of distance, health resource, and affordability between origins and destinations
## distance: railway shortest distance matrix estimated by China 1:100 million public dataset
## health resource: number of hospital beds and clinics in each city
## affordability: GDP per capita in each city
## author: Shiyu Deng
## data: January 2026
## affiliation: University College London (UCL)
## email: shiyu.deng.23@ucl.ac.uk


##------ Calculate GDP matrix --------

import pandas as pd
import numpy as np
import glob
import os

# === 1) Set folder paths ===
input_folder = "/weighted_gdp"
output_folder = "/gdp_matrix"
os.makedirs(output_folder, exist_ok=True)

def coerce_numeric(series: pd.Series) -> pd.Series:
    """
    Force a column containing commas, spaces, and possible units or
    miscellaneous characters to float.
    Invalid values are converted to NaN.
    """
    s = series.astype(str)
    # Remove thousand separators and spaces
    s = s.str.replace(',', '', regex=False).str.replace(' ', '', regex=False)
    # Extract a single numeric value (supports scientific notation)
    s = s.str.extract(r'([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?)', expand=False)
    return pd.to_numeric(s, errors='coerce')


# === 2) Loop over all CSV files ===
for file in glob.glob(os.path.join(input_folder, "*.csv")):
    print("Processing:", file)
    
    # Read data
    df = pd.read_csv(file)
    df = df.rename(columns=str.lower)   # Standardize column names to lowercase
    if "city" not in df.columns or "pc_gdp" not in df.columns:
        print("⚠️ File missing 'city' or 'pc_gdp' column, skipped:", file)
        continue

    # Handle alternative naming (kept for robustness)
    city_col = "city" if "city" in df.columns else None
    gdp_col  = "pc_gdp" if "pc_gdp" in df.columns else None
    
    # Basic cleaning
    df = df[[city_col, gdp_col]].rename(columns={city_col: "city", gdp_col: "pc_gdp"})
    df["city"] = df["city"].astype(str).str.strip()
    df["pc_gdp"] = coerce_numeric(df["pc_gdp"])
    df = df.dropna(subset=["city", "pc_gdp"])
    print(df[df["pc_gdp"] <= 0])

    gdp = df["pc_gdp"].to_numpy()
    cities = df["city"].to_numpy()

    # Compute pairwise GDP ratio matrix
    matrix = gdp[None, :] / gdp[:, None]
    reject_df = pd.DataFrame(matrix, index=cities, columns=cities)
    np.fill_diagonal(reject_df.values, np.nan)
    
    # Output file name
    base = os.path.basename(file).replace(".csv", "")
    out_csv1 = os.path.join(output_folder, f"{base}_reject_matrix.csv")

    # Save results
    reject_df.to_csv(out_csv1)
    print(f"✅ Matrix saved: {out_csv1}")


In [ ]:
##------ Calculate city-level medical resource representative matrix --------

import pandas as pd
import numpy as np
import os

# ========= Configuration =========
INPUT_FILE = (
    "/National hospital directory.xlsx"
)   # Update to your file path

OUTPUT_FILE = (
    "/city_medical_resources.xlsx"
)   # Output Excel file

df = pd.read_excel(INPUT_FILE, sheet_name="destination hospital")
print(df.columns.tolist())

df_use = df[["city", "hospital name", "beds", "clinics"]].copy()

# Convert to numeric
df_use["beds"] = pd.to_numeric(df_use["beds"], errors="coerce")
df_use["clinics"] = pd.to_numeric(df_use["clinics"], errors="coerce")
# Drop missing or invalid values
df_use = df_use.dropna(subset=["city", "beds", "clinics"])
df_use = df_use[(df_use["beds"] > 0) & (df_use["clinics"] > 0)]

# ========= Hospital-level resource value = log(beds) × log(outpatient visits) =========
df_use["hospital_resource_value"] = df_use["beds"] * df_use["clinics"]
df_use["hospital_resource_value_log"] = (
    np.log(df_use["beds"]) * np.log(df_use["clinics"])
)

# ========= Aggregate to city level =========
agg = (
    df_use.groupby("city", as_index=False)
    .agg(
        hospital_count=("hospital name", "count"),
        total_beds=("beds", "sum"),
        total_clinics=("clinics", "sum"),
        resource_value=("hospital_resource_value", "sum"),
        resource_value_log=("hospital_resource_value_log", "sum"),
    )
)

# ========= Output =========
with pd.ExcelWriter(OUTPUT_FILE, engine="xlsxwriter") as writer:
    df_use.to_excel(writer, sheet_name="Hospital_Details", index=False)
    agg.sort_values("resource_value", ascending=False).to_excel(
        writer, sheet_name="City_Summary", index=False
    )


In [ ]:
## calculate railway shortest distance matrix between cities 

!pip install rasterio geopandas networkx numpy

import os
import numpy as np
import rasterio
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point
from math import sqrt
from tqdm import tqdm
import pandas as pd
from collections import defaultdict

# ========= Configuration =========
rail_tif = "/railroad_c.tif"              # Railway raster (must be GeoTIFF in a projected CRS with meter units)
cities_file = "/shi_en.shp"               # City point shapefile (CSV is also possible, see note below)
city_name_col = "City"                    # City name field
use_8_conn = True                         # Use 8-neighbor (diagonal) connectivity
rail_values = {1}                         # Raster values representing railway cells (adjust for your data)
search_radius = 50                        # Max search radius (in pixels) to snap cities to nearest railway cell
out_csv_long = "/rail_distance_pairs.csv"     # Long table: source, target, distance_km
out_csv_wide = "/rail_distance_matrix.csv"    # Wide table: distance matrix (rows/cols = cities)

os.makedirs("out", exist_ok=True)

# ========= Read railway raster =========
with rasterio.open(rail_tif) as src:
    rail = src.read(1)
    transform = src.transform
    crs = src.crs
    px = abs(transform.a)
    py = abs(transform.e)
    step = (px + py) / 2.0

if crs.is_geographic:
    raise ValueError(
        "rail.tif is in geographic coordinates (degrees). "
        "Please reproject it to a projected CRS in meters (e.g. EPSG:3857 or EPSG:4547) before running."
    )

# ========= Railway mask =========
mask = np.isin(rail, list(rail_values))
idx_r, idx_c = np.where(mask)
if len(idx_r) == 0:
    raise ValueError("No railway cells found. Please check rail_values.")

# ========= Build railway graph =========
G = nx.Graph()
dirs4 = [(-1, 0), (1, 0), (0, -1), (0, 1)]
dirs8 = dirs4 + [(-1, -1), (-1, 1), (1, -1), (1, 1)]
dirs = dirs8 if use_8_conn else dirs4

def arc_w(dr, dc):
    return step * (sqrt(2) if abs(dr) == 1 and abs(dc) == 1 else 1.0)

rail_bool = mask
H, W = rail.shape

# Only iterate over railway cells and connect neighboring railway cells
for i, j in zip(idx_r, idx_c):
    node = (i, j)
    G.add_node(node)
    for dr, dc in dirs:
        ni, nj = i + dr, j + dc
        if 0 <= ni < H and 0 <= nj < W and rail_bool[ni, nj]:
            G.add_edge(node, (ni, nj), weight=arc_w(dr, dc))

print(f"Graph built: nodes={G.number_of_nodes()}, edges={G.number_of_edges()}")


cities = gpd.read_file(cities_file)
if cities.crs is None:
    raise ValueError("City data has no CRS. Please make sure a coordinate system is defined.")

# Reproject cities to match railway CRS (must be in meters)
cities = cities.to_crs(crs)

# ========= Coordinate <-> raster index conversion =========
def xy_to_rowcol(x, y, transform):
    col, row = ~transform * (x, y)
    return int(round(row)), int(round(col))

# ========= Find nearest railway cell =========
def nearest_rail_cell(r, c, mask, max_search=50):
    if 0 <= r < mask.shape[0] and 0 <= c < mask.shape[1] and mask[r, c]:
        return (r, c)
    for rad in range(1, max_search + 1):
        rmin, rmax = max(0, r - rad), min(mask.shape[0] - 1, r + rad)
        cmin, cmax = max(0, c - rad), min(mask.shape[1] - 1, c + rad)
        sub = mask[rmin:rmax + 1, cmin:cmax + 1]
        if sub.any():
            rr, cc = np.where(sub)
            rr0, cc0 = rr[0] + rmin, cc[0] + cmin
            return (rr0, cc0)
    return None

city_nodes = {}
for idx, row in cities.iterrows():
    name = str(row[city_name_col])
    x, y = row.geometry.x, row.geometry.y
    r, c = xy_to_rowcol(x, y, transform)
    node = nearest_rail_cell(r, c, rail_bool, max_search=search_radius)
    if node is None:
        print(f"[Warning] No railway cell found near city {name} (radius={search_radius}). Skipped.")
    else:
        city_nodes[name] = node

valid_names = list(city_nodes.keys())
print(f"Cities used in calculation: {len(valid_names)} / {len(cities)}")

# ========= Multi-source shortest paths (Dijkstra per city) =========
pairs = []  # (source, target, distance_km)

for s in tqdm(valid_names, desc="Dijkstra by city"):
    src_node = city_nodes[s]
    dist_dict = nx.single_source_dijkstra_path_length(
        G, src_node, weight="weight"
    )
    for t in valid_names:
        if t == s:
            continue
        dst_node = city_nodes[t]
        d = dist_dict.get(dst_node)
        if d is not None:
            pairs.append((s, t, d / 1000.0))  # meters -> kilometers

# ========= Output: long table =========
df_long = pd.DataFrame(
    pairs, columns=["source_city", "target_city", "distance_km"]
)
df_long.to_csv(out_csv_long, index=False, encoding="utf-8-sig")
print(f"Long table saved to: {out_csv_long}")

# ========= Optional: symmetric distance matrix =========
cities_sorted = sorted(valid_names)
mat = pd.DataFrame(np.nan, index=cities_sorted, columns=cities_sorted)

for s, t, d in pairs:
    mat.loc[s, t] = d
    mat.loc[t, s] = d  # assume undirected / symmetric

np.fill_diagonal(mat.values, 0.0)
mat.to_csv(out_csv_wide, encoding="utf-8-sig")
print(f"Distance matrix saved to: {out_csv_wide}")

In [ ]:
##------ Calculate truncated power-law kernel matrix --------

import pandas as pd
import numpy as np

# ===== Parameters =====
INPUT_FILE = "/rail_distance_matrix.csv"   # Distance matrix file
SHEET_NAME = 0
BETA = 1.81
KPAR = 1758.5
TAIL_ONLY = False     # True = apply truncation only for r >= R_MIN; False = apply to all distances

# ===== 1) Read distance matrix D =====
D = pd.read_csv(INPUT_FILE, index_col=0).astype(float)
np.fill_diagonal(D.values, np.nan)

# ===== 2) Compute truncated power-law kernel K =====
with np.errstate(divide='ignore', invalid='ignore'):
    K = (D ** (-BETA))

if TAIL_ONLY:
    K = K * np.exp(-D / KPAR)

# ===== 3) Save results =====
K.to_csv("/kernel_trunc_powerlaw_matrix.csv", float_format="%.6g")

print(
    "✅ Completed:",
    "\n- Kernel matrix saved as: kernel_trunc_powerlaw_matrix.csv"
)

✅ 已保存: /Volumes/UCL/论文工作/空气污染/health_burden/air_fer_rcp_scenarios_45/city_patient_sum/citysum_cleanair_low_RCP2_6_2030.csv
✅ 已保存: /Volumes/UCL/论文工作/空气污染/health_burden/air_fer_rcp_scenarios_45/city_patient_sum/citysum_cleanair_low_RCP2_6_2040.csv
✅ 已保存: /Volumes/UCL/论文工作/空气污染/health_burden/air_fer_rcp_scenarios_45/city_patient_sum/citysum_cleanair_low_RCP2_6_2050.csv
✅ 已保存: /Volumes/UCL/论文工作/空气污染/health_burden/air_fer_rcp_scenarios_45/city_patient_sum/citysum_cleanair_low_RCP2_6_2060.csv
✅ 已保存: /Volumes/UCL/论文工作/空气污染/health_burden/air_fer_rcp_scenarios_45/city_patient_sum/citysum_cleanair_low_RCP4_5_2030.csv
✅ 已保存: /Volumes/UCL/论文工作/空气污染/health_burden/air_fer_rcp_scenarios_45/city_patient_sum/citysum_cleanair_low_RCP4_5_2040.csv
✅ 已保存: /Volumes/UCL/论文工作/空气污染/health_burden/air_fer_rcp_scenarios_45/city_patient_sum/citysum_cleanair_low_RCP4_5_2050.csv
✅ 已保存: /Volumes/UCL/论文工作/空气污染/health_burden/air_fer_rcp_scenarios_45/city_patient_sum/citysum_cleanair_low_RCP4_5_2060.csv
✅ 已保存: /Volumes/

In [ ]:
##---- Compute cross-city patient flows ----------
import pandas as pd
import numpy as np
import os, glob

# ===== Paths and templates (as provided) =====
DIST_FILE = "kernel_trunc_powerlaw_matrix_截断.csv"  # rows/cols = cities

S_FILE    = "/city_medical_resources.xlsx"  # contains city and resource column
S_COL     = "resource_value"

CITYSUM_DIR = "city_patient_sum"  # includes stroke after filtering
CITYSUM_TPL = "citysum_{air}_{fer}_{rcp}_{year}.csv"

G_DIR  = "gdp_matrix"
G_TPL  = "city_pop_weighted_gdp_{year}_{fer}_{rcp}_reject_matrix.csv"  # order: year, fer, rcp

OUT_DIR = "/cross_flow"
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUT_DIR, "aij_results"), exist_ok=True)
os.makedirs(os.path.join(OUT_DIR, "flow_results"), exist_ok=True)

# ===== Scenario sets =====
air_scenarios = ['ref', 'cleanair', 'earlypeak_NZ_CL', 'ontimepeak_CL', 'ontimepeak_NZ_CL']
fer_scenarios = ['low', 'mid', 'high']
rcp_scenarios = ['RCP2_6', 'RCP4_5', 'RCP8_5']
years = list(range(2020, 2061, 10))


def read_S_from_excel(xlsx_path: str, value_col: str) -> pd.DataFrame:
    """Read a table containing columns 'city' and value_col from any sheet in an Excel file."""
    xls = pd.ExcelFile(xlsx_path)
    for sh in xls.sheet_names:
        df = pd.read_excel(xlsx_path, sheet_name=sh)
        cols = [c.strip() if isinstance(c, str) else c for c in df.columns]
        df.columns = cols
        if "city" in df.columns and value_col in df.columns:
            out = df[["city", value_col]].copy()
            out["city"] = out["city"].astype(str).str.strip()
            return out
    raise ValueError(f"Could not find columns 'city' and '{value_col}' in any sheet of {xlsx_path}")

# ===== Fixed inputs =====
D_full = pd.read_csv(DIST_FILE, index_col=0).astype(float)

if not np.isnan(np.diag(D_full.values)).all():
    np.fill_diagonal(D_full.values, np.nan)

# Medical resources (168 target cities)
S_df = read_S_from_excel(S_FILE, S_COL)
targets_168 = S_df["city"].tolist()
S_full = S_df.set_index("city")[S_COL].astype(float)

# ===== Main loop =====
for air in air_scenarios:
    for fer in fer_scenarios:
        for rcp in rcp_scenarios:
            for year in years:

                # --- V: define sources based on citysum (360) ---
                v_path = os.path.join(CITYSUM_DIR, CITYSUM_TPL.format(air=air, fer=fer, rcp=rcp, year=year))
                v_files = glob.glob(v_path)
                if not os.path.exists(v_path):
                    print(f"⚠️ V file not found: {v_path}")
                    continue  # skip this iteration

                Vdf = pd.read_csv(v_files[0])
                city_col = "city" if "city" in Vdf.columns else Vdf.columns[0]
                vcol = "local_patient"
                Vdf[city_col] = Vdf[city_col].astype(str).str.strip()
                sources_360 = Vdf[city_col].tolist()

                # --- G: read and align to (sources × targets) ---
                g_path = os.path.join(G_DIR, G_TPL.format(year=year, fer=fer, rcp=rcp))
                g_files = glob.glob(g_path)
                if not os.path.exists(g_path):
                    print(f"⚠️ G file not found: {g_path}")
                    continue  # skip this iteration

                G_full = pd.read_csv(g_files[0], index_col=0)

                # Align: rows = intersection of (citysum 360) with G and D;
                #        cols = intersection of (targets 168) with G and D
                rows = [c for c in sources_360 if c in G_full.index and c in D_full.index]
                cols = [c for c in targets_168 if c in G_full.columns and c in D_full.columns]
                print(f"row cities (source): {rows} | col cities (targets): {cols}")

                # --- V_ratio: V_i / V_j ---
                lp_series = Vdf.set_index(city_col)[vcol].astype(float)
                LP_rows = lp_series.reindex(rows)
                LP_cols = lp_series.reindex(cols)
                V_ratio = LP_rows.values[:, None] / LP_cols.values[None, :]
                print("V_ratio", V_ratio)

                G = G_full.loc[rows, cols].copy()
                D = D_full.loc[rows, cols].copy()

                # --- S_j: keep only target cities ---
                S_vec = S_full.reindex(cols).astype(float).values

                # --- Compute a_ij = (G_ij * S_j * D_ij * V_ratio) ---
                # Note: The current implementation below does NOT multiply by V_ratio.
                A = G.values * S_vec[None, :] * D.values
                A = pd.DataFrame(A, index=rows, columns=cols)

                # --- Row-normalize by j for each i (row normalization) ---
                row_sum = A.sum(axis=1).replace(0, np.nan)
                A_rowprob = A.div(row_sum, axis=0).fillna(0.0)

                # --- Save a_ij ---
                tag = f"{air}_{fer}_{rcp}_{year}"
                A_rowprob.to_csv(
                    os.path.join(OUT_DIR, f"aij_results/aij_rowprob_{tag}.csv"),
                    float_format="%.8g"
                )
                print(f"✅ a_ij matrix saved: {tag}  | shape={A_rowprob.shape}")

                # === Compute the actual flow matrix F_ij (number of patients) ===
                C_vec = Vdf.set_index(city_col)["cross_patient_total"].astype(float).reindex(rows)
                F = A_rowprob.mul(C_vec, axis=0)
                F.to_csv(
                    os.path.join(OUT_DIR, f"flow_results/flow_patientnum_{tag}.csv"),
                    float_format="%.8g"
                )
                print(f"✅ Patient flow matrix saved: {tag}  | shape={F.shape}")


⚠️ 找不到 G 文件: /Volumes/UCL/论文工作/空气污染/weighted_gdp/gdp_matrix/city_pop_weighted_gdp_2020_low_RCP2_6_reject_matrix.csv
row cities (source): ['Aba', 'Akesu', 'Alaer', 'Alashan', 'Aletai', 'Ali', 'Ankang', 'Anqing', 'Anshan', 'Anshun', 'Anyang', 'Baicheng', 'Baise', 'Baisha', 'Baishan', 'Baiyin', 'Baoding', 'Baoji', 'Baoshan', 'Baoting', 'Baotou', 'Bayanzhuoer', 'Bayinguoleng', 'Bazhong', 'Beihai', 'Beijing', 'Bengbu', 'Benxi', 'Bijie', 'Binnzhou', 'Binzhou', 'Boertalameng', 'Bozhou', 'Cangzhou', 'Changchun', 'Changde', 'Changdu', 'Changji', 'Changjiang', 'Changsha', 'Changzhi', 'Changzhou', 'Chaoyang', 'Chaozhou', 'Chengde', 'Chengdu', 'Chengmai', 'Chifeng', 'Chizhou', 'Chongqing', 'Chongzuo', 'Chuxiong', 'Chuzhou', 'Dali', 'Dalian', 'Dandong', 'Danzhou', 'Daqing', 'Datong', 'Daxinganling', 'Dazhou', 'Dehong', 'Deyang', 'Dezhou', 'Dingan', 'Dingxi', 'Diqing', 'Dongfang', 'Dongguan', 'Dongying', 'Eerduosi', 'Enshi', 'Ezhou', 'Fangchenggang', 'Foshan', 'Fushun', 'Fuuzhou', 'Fuxin', 'Fuyang',